In [1]:
import pandas as pd
import get_data, analysis, utils, plotting, compute_comethylation, methyl_mut_burden
import os
import matplotlib.pyplot as plt

In [7]:
dependency_f_dir = "./dependency_files"
data_dir = "./data"
mut_fn = os.path.join(data_dir, "PANCAN_mut.tsv.gz")


In [8]:
# reaad in mutations
all_mut_df = get_data.get_mutations(mut_fn)
all_mut_df['mutation'] = all_mut_df['chr'] + ':' + all_mut_df['start'].astype(str)


### GoDMC

In [2]:
# read in goDMC meQTLs
meqtl_df = pd.read_parquet("/cellar/users/zkoch/methylation_and_mutation/data/meQTL/goDMC_meQTL/goDMC_meQTLs.parquet")

In [13]:
# split snp column on ':' and expand to get chr, start, and type
meqtl_df[['chr', 'start', 'type']] = meqtl_df['snp'].str.split(':', expand=True)
# remove 'chr' from chr column
meqtl_df['chr'] = meqtl_df['chr'].str.replace('chr', '')
# create mutation column
meqtl_df['mutation'] = meqtl_df['chr'] + ':' + meqtl_df['start']

KeyboardInterrupt: 

In [ ]:
# merge on mutation to get all mutations that are meQTLs
shared_mut_df = pd.merge(all_mut_df, meqtl_df, on='mutation', how='inner')

In [32]:
print("{} of {} somatic mutations are in the goDMC meqtls".format(len(shared_mut_df), len(all_mut_df)))

2837 of 2780312 somatic mutations are in the goDMC meqtls


In [33]:
2837/2780312

0.0010203890786357791

### tcga meqtl

In [5]:
tcga_meqtl_df = pd.read_csv("/cellar/users/zkoch/methylation_and_mutation/data/meQTL/pancan_tcga_meQTL/pancan_meQTL.tsv", sep='\t')

In [10]:
# rm 'chr' from snp_position column
tcga_meqtl_df['mutation'] = tcga_meqtl_df['snp_position'].str.replace('chr', '')
# merge on mutation to get all mutations that are meQTLs
tcga_merged = pd.merge(all_mut_df, tcga_meqtl_df, on='mutation', how='inner')

In [16]:
tcga_merged

,sample,chr,start,end,reference,alt,DNA_VAF,mutation,cancer_type,snp,snp_position,alleles,probes,probe_position,probe_gene,beta,status,r,p-value
0,TCGA-06-A5U1,1,152749036,152749036,G,T,0.24,1:152749036,LAML,rs149277953,chr1:152749036,GGGTGGT/G,cg24304714,chr1:152779374,LCE1C,-1.02,-4.57,-0.42,1.430000e-05
1,TCGA-13-2059,12,12482876,12482876,C,T,0.14,12:12482876,BLCA,rs113872597,chr12:12482876,CCTG/C,cg08615371,chr12:12503544,MANSC1,0.90,8.24,0.39,2.770000e-15
2,TCGA-13-2059,12,12482876,12482876,C,T,0.14,12:12482876,CESC,rs113872597,chr12:12482876,CCTG/C,cg08615371,chr12:12503544,MANSC1,0.87,6.19,0.36,2.320000e-09
3,TCGA-13-2059,12,12482876,12482876,C,T,0.14,12:12482876,ESCA,rs113872597,chr12:12482876,CCTG/C,cg08615371,chr12:12503544,MANSC1,0.77,4.88,0.37,2.740000e-06
4,TCGA-13-2059,12,12482876,12482876,C,T,0.14,12:12482876,HNSC,rs113872597,chr12:12482876,CCTG/C,cg08615371,chr12:12503544,MANSC1,0.95,10.67,0.44,5.650000e-24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,TCGA-VR-A8EY,16,87738888,87738888,G,T,0.50,16:87738888,TGCT,rs12447897,chr16:87738888,G/T,cg07873172,chr16:87712631,JPH3,0.41,4.55,0.38,1.280000e-05
251,TCGA-XE-AAOL,1,11982373,11982373,A,G,0.59,1:11982373,THCA,rs1137777,chr1:11982373,A/G,cg00299820,chr1:11943154,NaN,0.37,7.35,0.32,8.980000e-13
252,TCGA-XU-A936,12,125549956,125549956,C,T,0.64,12:125549956,LGG,rs41474647,chr12:125549956,C/T,cg03013422,chr12:125550350,AACS,0.52,7.67,0.34,1.030000e-13
253,TCGA-XU-A936,12,125549956,125549956,C,T,0.64,12:125549956,LGG,rs41474647,chr12:125549956,C/T,cg21265540,chr12:125534110,NaN,0.47,6.67,0.30,7.350000e-11
